# Importowanie bibliotek
W projekcie wykorzystano następujące biblioteki:
- TensorFlow i Keras: Do budowy i trenowania modelu głębokiego uczenia.
- NumPy i Pandas: Do manipulacji danymi i ich wstępnego przetwarzania.
- Scikit-learn: Do skalowania danych oraz podziału na zbiory treningowe i testowe.
- Keras Tuner: Do optymalizacji hiperparametrów modelu.
- Callbacks w Keras: EarlyStopping i ReduceLROnPlateau zapewniają zatrzymanie treningu w odpowiednim momencie i dostosowywanie tempa uczenia.

In [8]:
import keras_tuner as kt
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import tensorflow as tf

# Wczytanie i przygotowanie danych
Źródło danych: Dane zostały wczytane z pliku CSV (data.csv).

Filtracja danych: Wybrano tylko numeryczne kolumny istotne dla klasyfikacji:
- Size(L) - rozmiar partii piwa.
- OG, FG - gęstość początkowa i końcowa.
- ABV - zawartość alkoholu (%).
- IBU - jednostki goryczki.
- Color - barwa piwa.
- BoilSize, BoilTime, BoilGravity, Efficiency - parametry procesu warzenia.
Usuwanie braków danych: Wszystkie wiersze zawierające braki danych zostały usunięte.

Kodowanie etykiet: Style piwa zakodowano jako wartości całkowite przy użyciu LabelEncoder.

Skalowanie danych: Znormalizowano dane wejściowe za pomocą StandardScaler, aby ułatwić proces uczenia.

In [2]:
data = pd.read_csv('../data.csv')
numeric_columns_classification = ['Size(L)', 'OG', 'FG', 'ABV', 'IBU', 'Color', 'BoilSize', 'BoilTime', 'BoilGravity', 'Efficiency']
data[numeric_columns_classification] = data[numeric_columns_classification].apply(pd.to_numeric, errors='coerce')
data = data.dropna()
X = data[numeric_columns_classification]
y = data['Style']

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Struktura modelu
Zbudowano model sekwencyjny z następującymi warstwami:
- Warstwa wejściowa:
  - Liczba neuronów dynamicznie dobierana (32–512, krok 32).
  - Aktywacja ReLU.
- Batch Normalization: Stabilizacja procesu uczenia przez normalizację wsadową.
- Dropout: Regularizacja z dynamicznie dobieraną wartością (0–50%, krok 10%).
- Warstwy ukryte:
  - Pierwsza warstwa: Liczba neuronów dynamicznie dobierana (32–512, krok 32), aktywacja ReLU.
  - Druga warstwa: Jak wyżej.
  - Obie warstwy ukryte wyposażono w Batch Normalization i Dropout.
- Warstwa wyjściowa: Liczba neuronów równa liczbie klas (style piwa), aktywacja softmax.

Kompilacja modelu
- Optymalizator: Adam z dynamicznie dobieraną szybkością uczenia (0.0001–0.01, skala logarytmiczna).
- Funkcja straty: sparse_categorical_crossentropy (dla zakodowanych liczb całkowitych).
- Metryka: accuracy (dokładność).

In [3]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_shape=(X_scaled.shape[1],)))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Proces tuningu hiperparametrów
Metoda tuningu: Bayesian Optimization (optymalizacja bayesowska).
- Hiperparametry:
  - Liczba neuronów w warstwach: 32–512 (krok 32).
  - Dropout: 0–50% (krok 10%).
  - Learning rate: 0.0001–0.01 (skala logarytmiczna).
- Liczba prób: 20 (maksymalna liczba konfiguracji testowych).

Proces uczenia
- Parametry treningu:
  - Maksymalna liczba epok: 20.
  - Batch size: 32.
- Callbacki:
  - EarlyStopping: Monitorowanie straty walidacji, zatrzymanie po 5 epokach bez poprawy.
  - TensorBoard: Monitorowanie wyników uczenia w czasie rzeczywistym.

In [4]:
tuner = kt.BayesianOptimization(build_model,
                                 objective='val_accuracy',
                                 max_trials=20, 
                                 directory='logs/fit', 
                                 project_name='klasyfikacja_styli_piwa')

early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=5, 
                               restore_best_weights=True)

log_dir = "logs/fit/klasyfikacja_styli_piwa" 
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

tuner.search(X_scaled, y_encoded, 
             epochs=20, 
             validation_split=0.2, 
             callbacks=[early_stopping, tensorboard_callback], 
             batch_size=32)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 20 Complete [00h 00m 03s]
val_accuracy: 0.28947368264198303

Best val_accuracy So Far: 0.31578946113586426
Total elapsed time: 00h 01m 23s


In [5]:
best_model = build_model(best_hps)

best_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 128)            │         1,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 119)            │        15,351 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,295 (133.96 KB)

 Trainable params: 33,783 (131.96 KB)

 Non-trainable params: 512 (2.00 KB)

# Wyniki
Najlepszy model: Osiągnął dokładność walidacyjną 31.57%.

Finalna dokładność walidacyjna: 28%.

Strata walidacyjna: Stabilizowała się w granicach 3.5–3.7 po kilkunastu epokach.

In [6]:
history = best_model.fit(X_scaled, y_encoded, 
                         epochs=20, 
                         validation_split=0.2, 
                         callbacks=[early_stopping, tensorboard_callback],
                         batch_size=32)

final_val_accuracy = history.history['val_accuracy'][-1]
print(f"Final Validation Accuracy: {final_val_accuracy:.2f}")

Epoch 1/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.1243 - loss: 4.5043 - val_accuracy: 0.2039 - val_loss: 4.0986
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.2969 - loss: 2.9427 - val_accuracy: 0.2039 - val_loss: 3.7107
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3327 - loss: 2.5237 - val_accuracy: 0.2237 - val_loss: 3.5992
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3701 - loss: 2.3685 - val_accuracy: 0.2039 - val_loss: 3.5978
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.3762 - loss: 2.2658 - val_accuracy: 0.2434 - val_loss: 3.5738
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4294 - loss: 2.0166 - val_accuracy: 0.2171 - val_loss: 3.6705
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4179 - loss: 1.9892 - val_accuracy: 0.2500 - val_loss: 3.5507
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.4442 - loss: 1.9035 - val_accuracy: 0.2500 - val_loss

In [7]:
%load_ext tensorboard
%tensorboard --logdir logs/fit/klasyfikacja_styli_piwa